### 製作新的 Synonym_base 
- 以 `<動詞片語, 受詞>` pair 的方式儲存一組介紹，動詞片語盡量取 lemma (原型動詞) 如 "copy a file"。
- 在手冊中每個 Syscall 的描述有 Name 區塊和 Description 區塊，每個 syscall 一定會有 Name 區塊來簡短的描述，如: `write - write to a file descriptor`。我們把 Name 區塊的 pair 稱作 `FirstLine` 欄位，理論上每個 syscall 都會有一個 `<動詞片語, 受詞>` pair 所以會有一行資料，若 First line 有兩個動詞 pair 則分開寫成兩行。
- Description 區塊是很長且非結構化的詳細敘述，說明 syscall 的運作方式、有哪些功能參數以及如何呼叫。我們盡量找 >= 1 組與 `FirstLine` 相同意義的句子，也製作成 pair 放在 `Synonym` 欄位。一個 `FirstLine` 可以有 0 或多個 synonym pair，通通寫在同一行用`"|"`分隔`<>`。
- 問題: pair 的受詞要多細膩? 回答:
```
check real user's permissions for a file => "<check,user's permission>"
```

Load origin rule_dataset, convert to pair format

In [56]:
import pandas as pd

# df = pd.read_csv('../rule_dataset_final.csv') # 354 Syscall
# df = pd.read_csv('../rule_dataset.csv')       # 88  Syscall
df = pd.read_csv('../proc.csv')       # 88  Syscall

def clean_braces(x):
    x = x.replace("()", "").replace(",", "").replace(" ", "")
    return str(x)
def add_braces(x):
    x = clean_braces(x)
    x = x+"()"
    return x

df['Syscall'] = df['Syscall'].apply(add_braces) # or clean_braces
df.head(3)


,ActionType,Syscall,EnVerb,Sentence,EntityType
0,PROC_WAIT,idle(),make,make process 0 idle,PROC
1,PROC_OTHER,move_pages(),move,move individual pages of a process to another ...,PROC
2,PROC_OTHER,timer_gettime(),fetch,arm/disarm and fetch state of POSIX per,PROC


In [57]:
def make_firstline(v:str, sentence:str, default="<,>"):
    spl = sentence.split()
    for i,word in enumerate(spl):
        if word.startswith(v):
            object_phrase = " ".join(spl[i+1:])
            return f"<{v},{object_phrase}>"
    return default

In [58]:
# New columns: FirstLine Synonym
# Better to remain columns: (EntityType ActionType) Syscall

# syscall_88 = df['Syscall'].unique()
# df = df[df['Source'] == 'man']
df['FirstLine'] = df.apply(lambda r: make_firstline(r['EnVerb'],r['Sentence']), axis=1)
df['Synonym']   = "<,>"
df['SynonymSentence']   = ""
df = df[['Syscall', 'Sentence', 'FirstLine', 'Synonym', 'SynonymSentence']]
df.to_csv('proc.csv', index=False)
df

,Syscall,Sentence,FirstLine,Synonym,SynonymSentence
0,idle(),make process 0 idle,"<make,process 0 idle>","<,>",
1,move_pages(),move individual pages of a process to another ...,"<move,individual pages of a process to another...","<,>",
2,timer_gettime(),arm/disarm and fetch state of POSIX per,"<fetch,state of POSIX per>","<,>",
3,timer_delete(),delete a POSIX per,"<delete,a POSIX per>","<,>",
4,execveat(),execute program relative to a directory file d...,"<execute,program relative to a directory file ...","<,>",
...,...,...,...,...,...
74,ioprio_set(),get/set I/O scheduling class and priority,"<,>","<,>",
75,sched_getattr(),set and get scheduling policy and attributes,"<get,scheduling policy and attributes>","<,>",
76,get_robust_list(),get/set list of robust futexes,"<get,list of robust futexes>","<,>",
77,getitimer(),get or set value of an interval timer,"<get,or set value of an interval timer>","<,>",


In [ ]:
'''
Syscall,Sentence,FirstLine,Synonym
read(),read from a file descriptor,"<read, file descriptor>","<>"
readlink(),read value of a symbolic link,"<read, value of a symbolic link>","<place, the contents of the symbolic link path>"
'''